# Tutorial 04: Run the Training and Test Phases of the WGAST Model

In this tutorial, we will run the **WGAST** model on the structured dataset we prepared earlier. This includes:

- Setting up all necessary parameters and model configurations.
- Launching the training process.
- Evaluating the trained model.

📌 This tutorial is fully customizable. The default parameters match the settings used in our paper, feel free to adjust them based on your data or hardware capabilities.


In [2]:
# === Imports ===
from pathlib import Path
import torch
import torch.backends.cudnn as cudnn
import time

import sys
import os
# Add the project root to sys.path to allow imports from other folders (e.g., 'runner', 'data_loader', etc.)
sys.path.append(os.path.abspath('..'))  # Assumes this notebook is in /tutorials/

from runner.experiment import Experiment  # Main class to manage training and testing


## Step 01 : Define and Initialize Model Parameters (from the Paper)

In [4]:
# === Configuration Class ===
class Options:
    """
    Stores all hyperparameters and paths used during training/testing.

    ⚠️ NOTE: The folder './data/Tdivision' must be generated by the previous tutorials:
        - 01_data_download.ipynb
        - 02_data_preparation.ipynb
        - 03_data_structuring.ipynb

    📌 These hyperparameters correspond to the settings used in the paper.
       Feel free to adjust them for your own experiments.
    """
    def __init__(self):
        # Training settings
        self.lr = 2e-4                  # Learning rate
        self.batch_size = 32           # Batch size
        self.epochs = 110              # Number of training epochs

        # Hardware settings
        self.cuda = True               # Enable CUDA if available
        self.ngpu = 1                  # Number of GPUs to use
        self.num_workers = 8           # Number of data loading workers

        # Data paths
        self.save_dir = Path('./data/Tdivision')      # Where to save outputs
        self.data_dir = Path('./data/Tdivision')      # Base data directory
        self.train_dir = Path('./data/Tdivision/train')  # Training data path
        self.test_dir = Path('./data/Tdivision/test')    # Testing data path

        # Image and patch parameters
        self.image_size = [400, 400]   # Size of input images
        self.patch_size = [32, 32]     # Size of image patches
        self.patch_stride = 8          # Stride for patch extraction
        self.test_patch = 32           # Patch size during testing

        # Model options
        self.ifAdaIN = True            # Use AdaIN for feature normalization
        self.ifAttention = True        # Use attention mechanism
        self.ifTwoInput = False        # Use two input streams (if applicable)
        
        # Loss weights (used in final objective function)
        self.a = 1e-2
        self.b = 1
        self.c = 1
        self.d = 1

# === Set Options ===
opt = Options()

# === Set random seed for reproducibility ===
torch.manual_seed(2024)
opt.cuda = True  # Enable CUDA if available
if not torch.cuda.is_available():
    opt.cuda = False

if opt.cuda:
    torch.cuda.manual_seed_all(2024)
    cudnn.benchmark = True       # Enable benchmark mode for optimal performance
    cudnn.deterministic = True   # Make training deterministic (reproducible)

# === Run Experiment ===
# This initializes the training/testing workflow
experiment = Experiment(opt)


Model initialization
There are 7884208 trainable parameters for generator.
There are 2765505 trainable parameters for nlayerdiscriminator.


## Step 02 : Train the Model

In [ ]:
if opt.epochs > 0:
    start_time = time.time()  # Start measuring training time
    
    # Begin training process using parameters defined earlier
    # - opt.train_dir: Path to the training dataset (prepared in previous tutorials)
    # - patch_size / patch_stride: How images are split into training patches
    # - batch_size: Number of samples per training batch
    # - num_workers: Number of subprocesses used for data loading
    # - epochs: Number of full passes over the dataset
    predictions = experiment.train(opt.train_dir,
                                   opt.patch_size, 
                                   opt.patch_stride, 
                                   opt.batch_size,
                                   num_workers=1, 
                                   epochs=opt.epochs)

    end_time = time.time()  # Stop measuring training time
    
    elapsed_time = end_time - start_time
    print(f"Training took {elapsed_time:.2f} seconds")  # Display training duration


## Step 03 : Test the Model

In [ ]:
# Evaluate the trained model using the test dataset
# - opt.test_dir: Path to the test dataset (should be pre-generated in earlier steps)
# - patch_size: Size of the patches used for testing
# - num_workers: Number of parallel data loading workers

results = experiment.test(opt.test_dir,
                          opt.patch_size,
                          num_workers=1)

# Print or log the results as needed
print("Testing completed.")
